In [1]:
### Mükellef Tableau Case Study ###

# Connect MS SQL Server with Python.
# Create Dataset as "mükellef" on MS SQL Server via Python.
# Load 4 csv Tables to dataset via MS SQL Server.
# Get tables with python.
# Use Left Join or Full Outer Join to get a unique table.
# Exploratory Data Analysis to make a clean data.
# Download new, clean unique table as csv to use on Tableau.

In [2]:
# pip install pyodbc

In [3]:
import pandas as pd
import pyodbc
import sqlalchemy
import datetime
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import warnings
warnings.filterwarnings('ignore')

### MS SQL Server Connection

In [4]:
conn_string = "driver={ODBC Driver 17 for SQL Server}; server=localhost; database=master; TRUSTED_CONNECTION=yes;"

In [5]:
conn = pyodbc.connect(conn_string);

In [6]:
crs = conn.cursor()

In [7]:
conn.autocommit = True

In [8]:
create_database_query = 'Create database mükellef'

In [9]:
def create_database (conn, create_database_query):
    crs = conn.cursor()
    try: 
        crs.execute(create_database_query)
        print("Database is Created Succesfully")
    except Error as err:
        print(f"Error:'{err}'")

In [10]:
# create_database(conn, create_database_query)

In [11]:
def execute_query(conn, query):
    crs = conn.cursor()
    try:
        crs.execute(query)
        conn.commit()
        print("Query Succeessful!")
    except Error as err:
        print(f"Error:'{err}'")

In [12]:
query = 'USE mükellef'
execute_query(conn, query)

Query Succeessful!


### Data Info

In [13]:
query = 'SELECT COUNT(*) FROM transaction_items'
execute_query(conn, query)

Query Succeessful!


In [14]:
query = 'SELECT * FROM transaction_items'
df1 = pd.read_sql(query, con = conn)
df1.head()

,Customer_ID,Products_ID,Price,Refund,Revenue,Share,Commission,Payment_Provider_Commission
0,67212,1,670.240,0.000,120.070,542.800,305.320,7.370
1,67207,1,609.290,0.000,127.640,474.950,223.900,6.700
2,67201,1,202.190,0.000,199.970,0.000,0.000,2.220
3,67197,1,609.290,0.000,127.640,474.950,223.900,6.700
4,67187,1,670.240,0.000,120.070,542.800,305.320,7.370


In [15]:
df1.duplicated().value_counts()

False    2383
True      416
dtype: int64

In [16]:
query = 'SELECT * FROM transactions'
df2 = pd.read_sql(query, con = conn)
df2.head()

,Customer_ID,ID,Price,Currency,Created_At
0,67212,111323,670.240,TRL,2022-08-19T17:16:26Z
1,67207,111313,609.290,TRL,2022-08-19T15:44:03Z
2,67201,111296,202.190,TRL,2022-08-19T13:33:25Z
3,67197,111321,609.290,TRL,2022-08-19T16:48:41Z
4,67187,111288,670.240,TRL,2022-08-19T11:34:03Z


In [17]:
df2.duplicated().value_counts()

False    1931
dtype: int64

In [18]:
query = 'SELECT * FROM customers'
df3 = pd.read_sql(query, con = conn)
df3.head()

,ID,Is_Active,Period,Created_At,First_Paid_At,Last_Paid_At,Type
0,61302,True,monthly,2022-05-28T12:22:27Z,2022-06-13T20:03:20Z,2022-08-13T10:12:04Z,2
1,61310,True,monthly,2022-05-28T13:31:00Z,2022-05-28T14:15:24Z,2022-08-05T10:13:13Z,2
2,61319,True,annual,2022-05-28T14:28:56Z,2022-06-06T11:47:57Z,2022-06-06T11:47:57Z,8
3,61323,True,monthly,2022-05-28T14:43:20Z,2022-05-28T15:00:20Z,2022-07-28T10:35:59Z,2
4,61326,True,monthly,2022-05-28T15:01:35Z,2022-05-31T11:23:35Z,2022-08-01T10:39:14Z,12


In [19]:
df3.duplicated().value_counts()

False    1000
dtype: int64

In [20]:
query = 'SELECT * FROM cancel_requests'
df4 = pd.read_sql(query, con = conn)
df4.head()

,Customer_ID,Status,Approval_At,Cancel_Reason,Cancel_Request_At,Updated_At
0,66982,waiting_approval,NaT,0,2022-08-18T07:37:43.822Z,2022-08-18T10:37:44Z
1,66865,waiting_approval,NaT,0,2022-08-16T10:58:39.244Z,2022-08-16T15:03:34Z
2,66654,approved,2022-08-12 16:20:15,2,2022-08-12T11:16:16.686Z,2022-08-12T16:20:15Z
3,66538,waiting_approval,NaT,2,2022-08-18T14:27:24.788Z,2022-08-18T17:27:25Z
4,66526,waiting_approval,NaT,3,2022-08-19T12:51:24.158Z,2022-08-19T15:51:24Z


In [21]:
df4.duplicated().value_counts()

False    177
dtype: int64

In [22]:
query = "SELECT ti.Customer_ID, Products_ID, ti.Price AS TI_Price, Refund, Revenue, Share, Commission, Payment_Provider_Commission,\
                cu.Is_Active, Period, cu.Created_At AS CU_Created_At, First_Paid_At, Last_Paid_At, Type,\
                cr.Status, Approval_At, Cancel_Reason, Cancel_Request_At, Updated_At,\
                tr.ID AS TR_ID, tr.Price AS TR_Price, Currency, tr.Created_At AS TR_Created_At\
         FROM transaction_items ti\
         FULL OUTER JOIN customers cu ON ti.Customer_ID=cu.ID\
         FULL OUTER JOIN cancel_requests cr ON ti.Customer_ID=cr.Customer_ID\
         FULL OUTER JOIN transactions tr ON ti.Customer_ID=tr.Customer_ID"
df = pd.read_sql(query, con = conn)
df.head()

,Customer_ID,Products_ID,TI_Price,Refund,Revenue,Share,Commission,Payment_Provider_Commission,Is_Active,Period,...,Type,Status,Approval_At,Cancel_Reason,Cancel_Request_At,Updated_At,TR_ID,TR_Price,Currency,TR_Created_At
0,61302,1,474.950,0.000,130.480,339.250,0.000,5.220,True,monthly,...,2,None,NaT,NaN,None,None,99652,474.950,TRL,2022-06-13T20:03:20Z
1,61302,1,474.950,0.000,130.480,339.250,0.000,5.220,True,monthly,...,2,None,NaT,NaN,None,None,104884,474.950,TRL,2022-07-13T10:11:22Z
2,61302,1,474.950,0.000,130.480,339.250,0.000,5.220,True,monthly,...,2,None,NaT,NaN,None,None,110243,541.440,TRL,2022-08-13T10:12:04Z
3,61302,1,541.440,0.000,60.540,474.950,223.900,5.960,True,monthly,...,2,None,NaT,NaN,None,None,99652,474.950,TRL,2022-06-13T20:03:20Z
4,61302,1,541.440,0.000,60.540,474.950,223.900,5.960,True,monthly,...,2,None,NaT,NaN,None,None,104884,474.950,TRL,2022-07-13T10:11:22Z


In [23]:
df.shape

(6834, 23)

In [25]:
df.shape

(6834, 23)

In [26]:
print(len(df.columns))
print(df.columns)

23
Index(['Customer_ID', 'Products_ID', 'TI_Price', 'Refund', 'Revenue', 'Share',
       'Commission', 'Payment_Provider_Commission', 'Is_Active', 'Period',
       'CU_Created_At', 'First_Paid_At', 'Last_Paid_At', 'Type', 'Status',
       'Approval_At', 'Cancel_Reason', 'Cancel_Request_At', 'Updated_At',
       'TR_ID', 'TR_Price', 'Currency', 'TR_Created_At'],
      dtype='object')


In [27]:
df.duplicated().value_counts()

False    5569
True     1265
dtype: int64

In [28]:
df.drop_duplicates(inplace=True)

In [29]:
df.duplicated().value_counts()

False    5569
dtype: int64

In [30]:
df.head()

,Customer_ID,Products_ID,TI_Price,Refund,Revenue,Share,Commission,Payment_Provider_Commission,Is_Active,Period,...,Type,Status,Approval_At,Cancel_Reason,Cancel_Request_At,Updated_At,TR_ID,TR_Price,Currency,TR_Created_At
0,61302,1,474.950,0.000,130.480,339.250,0.000,5.220,True,monthly,...,2,None,NaT,NaN,None,None,99652,474.950,TRL,2022-06-13T20:03:20Z
1,61302,1,474.950,0.000,130.480,339.250,0.000,5.220,True,monthly,...,2,None,NaT,NaN,None,None,104884,474.950,TRL,2022-07-13T10:11:22Z
2,61302,1,474.950,0.000,130.480,339.250,0.000,5.220,True,monthly,...,2,None,NaT,NaN,None,None,110243,541.440,TRL,2022-08-13T10:12:04Z
3,61302,1,541.440,0.000,60.540,474.950,223.900,5.960,True,monthly,...,2,None,NaT,NaN,None,None,99652,474.950,TRL,2022-06-13T20:03:20Z
4,61302,1,541.440,0.000,60.540,474.950,223.900,5.960,True,monthly,...,2,None,NaT,NaN,None,None,104884,474.950,TRL,2022-07-13T10:11:22Z


In [31]:
df.shape

(5569, 23)

In [32]:
df.to_csv("mükellef.csv", index=False)

In [33]:
df = pd.read_csv("mükellef.csv")

In [35]:
df.head()

,Customer_ID,Products_ID,TI_Price,Refund,Revenue,Share,Commission,Payment_Provider_Commission,Is_Active,Period,...,Type,Status,Approval_At,Cancel_Reason,Cancel_Request_At,Updated_At,TR_ID,TR_Price,Currency,TR_Created_At
0,61302,1,474.950,0.000,130.480,339.250,0.000,5.220,True,monthly,...,2,NaN,NaN,NaN,NaN,NaN,99652,474.950,TRL,2022-06-13T20:03:20Z
1,61302,1,474.950,0.000,130.480,339.250,0.000,5.220,True,monthly,...,2,NaN,NaN,NaN,NaN,NaN,104884,474.950,TRL,2022-07-13T10:11:22Z
2,61302,1,474.950,0.000,130.480,339.250,0.000,5.220,True,monthly,...,2,NaN,NaN,NaN,NaN,NaN,110243,541.440,TRL,2022-08-13T10:12:04Z
3,61302,1,541.440,0.000,60.540,474.950,223.900,5.960,True,monthly,...,2,NaN,NaN,NaN,NaN,NaN,99652,474.950,TRL,2022-06-13T20:03:20Z
4,61302,1,541.440,0.000,60.540,474.950,223.900,5.960,True,monthly,...,2,NaN,NaN,NaN,NaN,NaN,104884,474.950,TRL,2022-07-13T10:11:22Z


In [34]:
df.shape

(5569, 23)